# &#128202; LTV Caculation

## Data read & setting
### Import Module

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

import matplotlib.pyplot as plt
import seaborn as sns

import lifetimes
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import summary_data_from_transaction_data

from datetime import date

### PC setting

In [2]:
pd.options.display.float_format = '{:.2f}'.format

### Data read

In [3]:
user_date = pd.read_csv("/Users/jaehwan/Desktop/송재환/DS_Project/LTV/user_date.csv", encoding="utf-8")
user_subscription = pd.read_csv("/Users/jaehwan/Desktop/송재환/DS_Project/LTV/user_subscription.csv", encoding="utf-8")
df = pd.merge(user_date, user_subscription, on = 'user_id', how = 'left')
df['date'] = pd.to_datetime(df['date'])

## EDA

![EDA](https://i.esdrop.com/d/AVQLAkLtLT/ZtrY7zzLyB.png)

## LTV 계산
### 구독 사용자의 LTV: 7,496원

#### 데이터 관찰 시점은 데이터의 최신일인 2021년 6월 30일로 설정

![fomular](https://i.esdrop.com/d/AVQLAkLtLT/rC3LVHiBIt.png)

In [4]:
# 관찰 시점
observation_date='2021-06-30'

In [5]:
# 구독 사용자 분류
s_df = df[df['sub_type'] != "f"]

# user와 일자별 접속일 확인을 위한 데이터 처리
s_df = pd.DataFrame(s_df.groupby(['user_id','date'])['sub_type'].nunique()).rename(columns={'sub_type':'count'})
s_df.reset_index(inplace=True)

s_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51807 entries, 0 to 51806
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   user_id  51807 non-null  int64         
 1   date     51807 non-null  datetime64[ns]
 2   count    51807 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.2 MB


In [6]:
# BG/NBD 모델 적용을 위한 데이터 summarizing
summary = summary_data_from_transaction_data(s_df, 'user_id', 'date', observation_period_end = observation_date)
summary.reset_index().head(5)

,user_id,frequency,recency,T
0,5093,6.00,26.00,28.00
1,9355,36.00,60.00,60.00
2,10343,29.00,59.00,59.00
3,37926,32.00,59.00,59.00
4,41893,38.00,60.00,60.00


In [9]:
# BG/NBD 모델 적용
bgf = BetaGeoFitter(penalizer_coef=0.1)
bgf.fit(summary['frequency'], summary['recency'], summary['T'])

<lifetimes.BetaGeoFitter: fitted with 1046 subjects, a: 0.06, alpha: 2.83, b: 0.70, r: 1.56>

In [10]:
# BG/NBD 모델 적용하여 관찰 시점 기준 생존 가능성(이탈 가능성) 확인 
summary['conditional_probability_alive'] = bgf.conditional_probability_alive(summary['frequency'], summary['recency'], summary['T'])
summary[summary['conditional_probability_alive'] < 0.01].head(5)

,frequency,recency,T,conditional_probability_alive
user_id,,,,
43385,14.00,25.00,149.00,0.00
71870,29.00,52.00,119.00,0.00
112409,16.00,27.00,149.00,0.00
118945,66.00,119.00,149.00,0.00
273727,76.00,119.00,149.00,0.00


#### 관찰 기간 동안의 이탈 고객 수의 합

In [11]:
import datetime

observation_date = datetime.datetime.strptime(observation_date, '%Y-%m-%d')

In [15]:
# 관찰일 기준 30일 동안의 고객 이탈을 P(alive)를 계산하여 확인
observation_date_pre = observation_date - timedelta(30)
churn_user_list = []

for i in range(1,30):

    summary_pre = summary_data_from_transaction_data(s_df, 'user_id', 'date', observation_period_end = observation_date_pre)
    summary_pre = summary_pre[summary_pre['frequency']>0]
    
    bgf = BetaGeoFitter(penalizer_coef=0.3)
    bgf.fit(summary_pre['frequency'], summary_pre['recency'], summary_pre['T'])
    
    summary_pre['conditional_probability_alive'] = bgf.conditional_probability_alive(summary_pre['frequency'], summary_pre['recency'], summary_pre['T'])
    
    churn_user_list.append(list(summary_pre[summary_pre['conditional_probability_alive'] < 0.01].reset_index()['user_id']))

    observation_date_pre = observation_date_pre + timedelta(1)

churn_count = len(churn_user_list[0])
    
for j in np.arange(0,len(churn_user_list)-1):
    set1 = set(churn_user_list[j])
    set2 = set(churn_user_list[j+1])
    
    if set2 - set1 != set():
        churn_count = churn_count + len(list(set2 - set1))
        
print(f"관찰 기간 동안의 이탈 고객 수의 합: {churn_count}명")

관찰 기간 동안의 이탈 고객 수의 합: 278명


#### 관찰 기간 동안의 고객 수의 합

In [17]:
observation_date_pre = observation_date - timedelta(30)
active_user_count = 0

for i in range(1,30):
    
    active_user_count = active_user_count+s_df[s_df['date'] == observation_date_pre].groupby('date')['user_id'].count().values
    
    observation_date_pre = observation_date_pre + timedelta(1)
    
print(f"관찰 시점 동안의 고객 수의 합: {list(active_user_count)[0]}명")

관찰 시점 동안의 고객 수의 합: 12504명


#### 연 구독 사용자의 LTV

In [30]:
print(f"- 구독 사용자의 LTV = 평균 수익 / 이탈률 ")
print(f"- 평균 수익: 5,000원")
print(f"- 관찰 시점을 토대로 계산된 이탈률: 30*({churn_count}/{list(active_user_count)[0]})")
print(f"- 구독 사용자의 LTV = {round(5000/(30*(churn_count/list(active_user_count)[0])))}원")

- 구독 사용자의 LTV = 평균 수익 / 이탈률 
- 평균 수익: 5,000원
- 관찰 시점을 토대로 계산된 이탈률: 30*(278/12504)
- 구독 사용자의 LTV = 7496원


### 무료 사용자의 LTV: 57원

#### 데이터 관찰 시점은 데이터의 최신일인 2021년 6월 30일로 설정

In [22]:
# 관찰 시점
observation_date='2021-06-30'

In [23]:
f_df = df[df['sub_type'] == "f"]

# 일별 사용자가 서로 다른 기기로 이용했으면 다른 사용으로 보고 광고 수익 계산에 활용한다.
f_df = pd.DataFrame(f_df.groupby(['user_id','date'])['device_id'].nunique()*10).rename(columns={'device_id':'revenue_per_days'})
f_df.reset_index(inplace=True)

In [24]:
summary = summary_data_from_transaction_data(f_df, 'user_id', 'date', monetary_value_col='revenue_per_days', observation_period_end='2021-03-31')
new_summary = summary[summary['monetary_value'] != 0]
new_summary.head(5)

,frequency,recency,T,monetary_value
user_id,,,,
7215,17.00,58.00,58.00,10.00
8253,13.00,28.00,28.00,10.00
8331,15.00,69.00,77.00,10.00
10699,9.00,25.00,58.00,10.00
14918,26.00,80.00,88.00,11.15


In [25]:
bgf = BetaGeoFitter(penalizer_coef=0.0001)
bgf.fit(new_summary['frequency'], new_summary['recency'], new_summary['T'])

ggf = GammaGammaFitter(penalizer_coef = 0.0001)
ggf.fit(new_summary['frequency'], new_summary['monetary_value'])

<lifetimes.GammaGammaFitter: fitted with 3836 subjects, p: 21.90, q: 34.85, v: 16.51>

In [26]:
new_summary['CLV'] = round(ggf.customer_lifetime_value(
    bgf,
    new_summary['frequency'],
    new_summary['recency'],
    new_summary['T'],
    new_summary['monetary_value'],
    time=1,
    discount_rate=0
),2)

In [27]:
print(f"- 무료 사용자의 LTV = RFM, BG/NBD, Gamma-Gamma 모델을 활용한 계산")
print(f"- 사용자당 일 평균 광고 수익: 10원")
print(f"- 무료 사용자의 LTV = {round(new_summary['CLV'].mean())}원")

- 무료 사용자의 LTV = RFM, BG/NBD, Gamma-Gamma 모델을 활용한 계산
- 사용자당 일 평균 광고 수익: 10원
- 무료 사용자의 LTV = 57원
